## Python Library for Neural Network

Basically, most of the neural network libraries in Python requires [Theano](https://github.com/Theano/Theano) as a core library. Here is few easy-to-install and easy-to-implement libraries for neural network:

- [Lasagne](https://github.com/Lasagne/Lasagne) (has very good documentation [here]( http://lasagne.readthedocs.org/en/latest/index.html))

Here are wrapper libraries for `Lasagne`:
- [scikit-neuralnetwork](https://github.com/aigamedev/scikit-neuralnetwork)
- [nolearn](https://github.com/dnouri/nolearn)

I would suggest starting with **Lasagne** then go for others. Below is how to define each network layers for Lasagne (example from Github)

```python
import lasagne
import theano
import theano.tensor as T

# create Theano variables for input and target minibatch
input_var = T.tensor4('X')
target_var = T.ivector('y')

# create a small convolutional neural network
from lasagne.nonlinearities import leaky_rectify, softmax
network = lasagne.layers.InputLayer((None, 3, 32, 32), input_var)
network = lasagne.layers.Conv2DLayer(network, 64, (3, 3),
                                     nonlinearity=leaky_rectify)
network = lasagne.layers.Conv2DLayer(network, 32, (3, 3),
                                     nonlinearity=leaky_rectify)
network = lasagne.layers.Pool2DLayer(network, (3, 3), stride=2, mode='max')
network = lasagne.layers.DenseLayer(lasagne.layers.dropout(network, 0.5),
                                    128, nonlinearity=leaky_rectify,
                                    W=lasagne.init.Orthogonal())
network = lasagne.layers.DenseLayer(lasagne.layers.dropout(network, 0.5),
                                    10, nonlinearity=softmax)
```